In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# ───────────────────────────────────────────────────────────────────────────────
# 1) Subcluster data
# ───────────────────────────────────────────────────────────────────────────────
# Loading Master Store Subcluster file
df_subcluster = pd.read_csv(r" ... csv")

# Select and rename the required columns for Master Store Subcluster
selected_columns = {
    'Location Code': 'Store_No',
    'Name': 'Name',
    'CLOSEDATE': 'CLOSEDATE',
    'Opening Date': 'Opening_Date',
    'State': 'State',
    'CODE': 'CODE (subcluster 1)',
    'CODE FY26 1': 'CODE FY26 1 (subcluster 2)',
    'CODE FY26 2': 'CODE FY26 2 (subcluster 3)'
}
df_subcluster_selected = df_subcluster[list(selected_columns.keys())].rename(columns=selected_columns)

# Ensure correct datetime format for Opening_Date and CLOSEDATE
df_subcluster_selected["Opening_Date"] = pd.to_datetime(df_subcluster_selected["Opening_Date"], errors="coerce")
df_subcluster_selected["CLOSEDATE"] = pd.to_datetime(df_subcluster_selected["CLOSEDATE"], errors="coerce")  
# errors = "coerce"= "If a value can't be converted to a date, don't raise an error—just set it to NaT (Not a Time)."

# Filtering outliers
# 1. Closed stores: where CLOSEDATE is NOT null (i.e. already closed)
closed_stores = df_subcluster_selected[df_subcluster_selected["CLOSEDATE"].notna()]["Store_No"].astype(str)

# 2. Not open yet: where Opening_Date is in the future
today = pd.Timestamp.today().normalize()
not_open_yet = df_subcluster_selected[df_subcluster_selected["Opening_Date"] > today]["Store_No"].astype(str)

# 3. Empty Name: where Name is missing or blank
empty_name = df_subcluster_selected[df_subcluster_selected["Name"].isna() | (df_subcluster_selected["Name"].str.strip() == "")]["Store_No"].astype(str)

# Combine all outlier Store_Nos and changing to integer
outliers = pd.concat([closed_stores, not_open_yet, empty_name]).unique().tolist()
outliers_int = set(int(x) for x in outliers)

# Remove outliers from main dataset
df_subcluster_clean = df_subcluster_selected[~df_subcluster_selected["Store_No"].astype(int).isin(outliers_int)]

# Creating new column "Days after Opening"
df_subcluster_clean['Days_after_Opening'] = (datetime.today() - df_subcluster_clean['Opening_Date']).dt.days

# Creating duplicated rows with different dates from opening date to now
# Create an empty list to hold the expanded rows
expanded_rows = []

# Iterate over each row
for _, row in df_subcluster_clean.iterrows():
    opening_date = row['Opening_Date']
    if pd.isna(opening_date):
        continue  # Skip rows with invalid dates
    
    today = datetime.today().date()
    date_range = pd.date_range(start=opening_date, end=today)
    
    for i, date in enumerate(date_range):
        new_row = row.to_dict()
        new_row['Date'] = date
        new_row['Days_after_Opening'] = i
        expanded_rows.append(new_row)

# Create the expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Obtain only 2022-12-25 to 2025-06-17 data (weather data only up until then)
expanded_df = expanded_df[
    (expanded_df["Date"] >= "2022-12-25") &
    (expanded_df["Date"] <  "2025-06-17")
]

# Ensure date formats match
expanded_df['Date'] = pd.to_datetime(expanded_df['Date'])

# Dropping "CLOSEDATE" column
expanded_df = expanded_df.drop(columns=["CLOSEDATE", "Name"])

# ───────────────────────────────────────────────────────────────────────────────
# 2) Sales data
# ───────────────────────────────────────────────────────────────────────────────
# Loading Daily Sales Data
daily_sales = pd.read_csv(r" ... csv")

# Renaming columns so it matches the other merged data
daily_sales.rename(columns={
    "Store No": "Store_No",
    "Net Amount": "Net_Amount"
}, inplace=True)

## Keep only rows where Store_No is purely numeric - V is for vending machines
# Clean, then filter
daily_sales["Store_No"] = daily_sales["Store_No"].astype(str).str.strip()

# Filter purely numeric store numbers
sales_stores = daily_sales[daily_sales["Store_No"].str.fullmatch(r"\d+")].copy()

# Convert to integer after filtering
sales_stores["Store_No"] = sales_stores["Store_No"].astype(int) 

# Convert date to datetime
sales_stores["Date"] = pd.to_datetime(sales_stores["Date"], dayfirst=True, errors="coerce")

# Changing / format to - format
sales_stores["Date"] = sales_stores["Date"].dt.strftime("%Y-%m-%d")

# Convert date to datetime
sales_stores["Date"] = pd.to_datetime(sales_stores["Date"]) #, dayfirst=True, errors="coerce"

# Change columns to desired object types
sales_stores["Net_Amount"] = sales_stores["Net_Amount"].astype(float)
sales_stores["TC"] = sales_stores["TC"].astype(int)

# Obtain only 2022-12-25 to 2025-06-17
sales_stores = sales_stores[
    (sales_stores["Date"] >= "2022-12-25") &
    (sales_stores["Date"] <  "2025-06-17")
]

# Remove outliers from dataset
sales_stores = sales_stores[~sales_stores["Store_No"].astype(int).isin(outliers_int)]

sales_stores = sales_stores[sales_stores["TC"] >= 50]

# ───────────────────────────────────────────────────────────────────────────────
# 3) Merge sales data to public subcluster data 
# ───────────────────────────────────────────────────────────────────────────────
# Making sure merged data Store_No is also int
expanded_df["Store_No"] = expanded_df["Store_No"].astype(int) 

# Joining to create the final df - Weather, Store & Clusters, Public Holiday & (finally) Sales
df_subcluster_sales = pd.merge(sales_stores[["Date", "Store_No", "Net_Amount", "TC"]], expanded_df[["Store_No","Opening_Date","State","CODE (subcluster 1)","CODE FY26 1 (subcluster 2)",
                                                                                                    "CODE FY26 2 (subcluster 3)","Days_after_Opening", "Date"]],on=["Date", "Store_No"],how="left")

# Flip the sign of the amount
df_subcluster_sales["Net_Amount"] *= -1

# Taking only rows with Days after Opening data (filtering out test transactions)
df_subcluster_sales = df_subcluster_sales[df_subcluster_sales["Days_after_Opening"].notna()]

# ───────────────────────────────────────────────────────────────────────────────
# 4) Weather data
# ───────────────────────────────────────────────────────────────────────────────
# Load Weather Data Updated
wx = pd.read_csv(r" ... csv")

# Standardising time, Date and Hour
wx["ts_utc"] = pd.to_datetime(wx["Timestamp"].str.strip(), format="%Y-%m-%d %H:%M:%S%z", utc=True)
wx["ts_sg"]  = wx["ts_utc"].dt.tz_convert("Asia/Singapore")
wx["Date"]   = wx["ts_sg"].dt.date
wx["Hour"]   = wx["ts_sg"].dt.hour

# Changing Store_No to integer
wx["Store_No"] = wx["Store No"].astype(int)

# Selecting only columns that we need
wx = wx[["Date","Hour","Store_No","Weather Code","Temperature (°C)"]]

# Obtain only 2022-12-25 to 2027-01-01 data (weather data only up until 2025-06-17)
wx = wx[
    (wx["Date"] >= pd.to_datetime("2022-12-25").date()) &
    (wx["Date"] <  pd.to_datetime("2027-01-01").date())
]

# Creating Column "Rain?"
# Define code groups as integers
valid_clear_codes = {0, 1, 2, 3}
rain_codes = {51, 53, 55, 61, 63, 65}

# Group and apply logic directly on integers
wx_daily = wx.groupby(["Store_No", "Date"], as_index=False).agg({
    "Temperature (°C)": "mean",
    "Weather Code": lambda codes: (
        "Yes" if any(code in rain_codes for code in codes)
        else " No" if all(code in valid_clear_codes for code in codes)
        else "Error"
    )
})

# Rename columns
wx_daily.rename(columns={
    "Weather Code": "Rain?",
    "Temperature (°C)": "Average Daily Temperature (°C)"
}, inplace=True)

wx_daily["Date"] = pd.to_datetime(wx_daily["Date"], format="%Y-%m-%d %H:%M:%S%z")

# Filtering out outliers
wx_daily = wx_daily[~wx_daily["Store_No"].astype(int).isin(outliers_int)].copy()

# ───────────────────────────────────────────────────────────────────────────────
# 5) Merging weather data on subcluster and sales data
# ───────────────────────────────────────────────────────────────────────────────
merged_df = pd.merge(df_subcluster_sales, wx_daily, on=['Store_No', 'Date'], how='left')

# Filter: Keep rows where store was already open on that date
merged_df = merged_df[merged_df["Date"] >= merged_df["Opening_Date"]]

# Optional: Drop rows where key store details are still missing
critical_columns = ["State", "CODE FY26 1 (subcluster 2)", "CODE FY26 2 (subcluster 3)", "Days_after_Opening"] 
#not "CODE (subcluster 1)" as contains some stores that opened during 2024/2025 when the labelling is not up to date
merged_df = merged_df.dropna(subset=critical_columns)

# ───────────────────────────────────────────────────────────────────────────────
# 6) Public holiday data
# ───────────────────────────────────────────────────────────────────────────────
# Load the public holiday data
public_holiday_df = pd.read_excel(r"C:\Users\chiaj\OneDrive\Desktop\FM Project\PH for non-PH.xlsx")

# Fix datetime formats
public_holiday_df['Date'] = pd.to_datetime(public_holiday_df['Date'])
public_holiday_df['Opening_Date'] = pd.to_datetime(public_holiday_df['Opening_Date'])  # Fix: not the same as Date

merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Opening_Date'] = pd.to_datetime(merged_df['Opening_Date'])

# Merge only on essential keys
merged_ph_weather_subcluster_sales_df = pd.merge(
    merged_df,
    public_holiday_df[['Date', 'Store_No', 'Name', 'Puasa', 'Public Holiday']],
    on=['Date', 'Store_No'],
    how='left'  # preserve all rows from your main dataset
)

# ───────────────────────────────────────────────────────────────────────────────
# 7) Merging public holiday data on weather, subcluster, sales data
# ───────────────────────────────────────────────────────────────────────────────
# New Day column
merged_ph_weather_subcluster_sales_df['Day'] = merged_ph_weather_subcluster_sales_df['Date'].dt.day_name()

# New Month column
merged_ph_weather_subcluster_sales_df['Month'] = merged_ph_weather_subcluster_sales_df['Date'].dt.month_name()

merged_ph_weather_subcluster_sales_df['Date'] = pd.to_datetime(merged_ph_weather_subcluster_sales_df['Date'])
merged_ph_weather_subcluster_sales_df.to_csv(r" ... csv", index=False)